In [ ]:
import os
import cv2
import json
import numpy as np
import pandas
import skimage
import shapefile
import matplotlib.pyplot as plt

import modules

### load data

In [ ]:
kenya_osm, kenya_sf = modules.data.load_shapefile("kenya")
kenya_geo = modules.data.load_geodata("kenya")
kenya_dat = pandas.DataFrame.merge(kenya_geo, kenya_osm, on="index")

kenya_filenames = set(modules.data.util.load_image_filenames("kenya", D=128))

In [ ]:
N = 5000

kenya_dat[kenya_dat["class"] == "major"]
kenya_dat[kenya_dat["class"] == "minor"]
kenya_dat[kenya_dat["class"] == "two-track"]
kenya_dat["valid"] = [f"{int(index)}_{int(road_id)}.npy" in kenya_filenames for index, road_id in enumerate(kenya_dat.values[:, 0])]

major = kenya_dat.iloc[np.random.choice(kenya_dat[np.logical_and(kenya_dat["class"] == "major", kenya_dat["valid"] == True)].index, size=N, replace=False)]
minor = kenya_dat.iloc[np.random.choice(kenya_dat[np.logical_and(kenya_dat["class"] == "minor", kenya_dat["valid"] == True)].index, size=N, replace=False)]
two_track = kenya_dat.iloc[np.random.choice(kenya_dat[np.logical_and(kenya_dat["class"] == "two-track", kenya_dat["valid"] == True)].index, size=N, replace=False)]

filenames = [f"{idx}_{int(df.loc[idx]['id'])}.npy" for df in [major, minor, two_track] for idx in df.index]
permutation = np.arange(len(filenames))
np.random.shuffle(permutation)

filenames = [filenames[i] for i in permutation]
images = np.array([np.load(os.path.join(modules.data.util.root(), "kenya", "kenya_128x128_images", fname)) for fname in filenames])
labels = permutation // N